In [5]:


##
### https://web.stanford.edu/~hastie/ElemStatLearn/

'''Prostate data info

Predictors (columns 1--8)

lcavol
lweight
age
lbph
svi
lcp
gleason
pgg45

outcome (column 9)

lpsa

train/test indicator (column 10)

This last column indicates which 67 observations were used as the 
"training set" and which 30 as the test set, as described on page 48
in the book.

There was an error in these data in the first edition of this
book. Subject 32 had a value of 6.1 for lweight, which translates to a
449 gm prostate! The correct value is 44.9 gm. We are grateful to
Prof. Stephen W. Link for alerting us to this error.

The features must first be scaled to have mean zero and  variance 96 (=n)
before the analyses in Tables 3.1 and beyond.  That is, if x is the  96 by 8 matrix
of features, we compute xp <- scale(x,TRUE,TRUE)

'''
import pandas as pd
import altair as alt
import seaborn as sns
import numpy as np
from numpy.testing import assert_almost_equal
import matplotlib.pyplot as plt
from scipy import stats
from functools import reduce
def repeatedly(f, n): 
    def composition(f,g):
        # "f after g" 
        return lambda x: f(g(x))
    return reduce(lambda ma, mma: composition(ma, mma), [f]*n)

df_unnormalized = pd.read_csv("https://web.stanford.edu/~hastie/ElemStatLearn/datasets/prostate.data", 
                 delim_whitespace=True).select_dtypes(include=['float', 'int'])

#df_unnormalized['ones'] = np.ones(df_unnormalized.shape[0])
print(df_unnormalized.shape)
df_unnormalized.head()

(97, 9)


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783
2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519
3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519
4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519
5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564


# Grokking the f-statistic and z-score

Do a linear regression with `p` features. drop a feature (which is similar to requiring that one of the coefficients is zero), and train another model. 

the _F-statistic_ measures the difference between these two models. 

In [20]:
class OLS: 
    def __init__(self, dat, dependent): 
        self.X, self.y = self.X_from_df(dat, dependent)
        self.names = self.X.columns
        self.N = self.X.shape[0]
        assert self.N==self.y.shape[0]
        self.XtX_inv = self.XtX_inv(self.X)
        self.beta_hat = self.beta_hat()
        for feat in self.X.drop('ones', axis=1).columns: 
            assert_almost_equal(self.X[feat].mean(), 0)
            assert self.X[feat].isna().sum()==0
        self.vs = np.diag(self.XtX_inv)
        self.p = self.X.shape[1]
        self.vari_esti = np.divide(1, self.N - self.p - 2) * self.RSS(self.beta_hat)
        self.zs = np.divide(self.beta_hat, self.vari_esti * np.sqrt(self.vs))

    def X_from_df(self, dat, dependent): 
        '''takes: dat and the name of the y feature
    
        returns: centered dataframe with ones
        '''
        ones = pd.DataFrame(np.ones(dat.shape[0]), columns=['ones']).shift(1)
        X = np.divide(dat.drop(dependent, axis=1) - dat.drop(dependent, axis=1).mean(),
                  dat.drop(dependent, axis=1).var())

        df = pd.concat([ones, X], axis=1).drop(0, axis=0)
        df.ones.iloc[df.shape[0]-1] = 1
        return df, dat[dependent]    
        
    def XtX_inv(self, X): 
        XtX = np.matmul(X.T, X)
        return np.linalg.inv(XtX)
    
    def beta_hat(self):
        '''analytic solution to OLS'''
        left_fact = np.matmul(self.XtX_inv, self.X.T)
        return np.matmul(left_fact, self.y)

    def RSS(self, beta):#, X=self.X, y=self.y): 
        '''residual sum squared'''
        resid_mat = self.y - np.matmul(self.X, beta)
        return np.matmul(resid_mat.T, resid_mat)

    def drop_one(self, i): 
        to_drop = self.names[i]
        return X.drop(to_drop, axis=1)
    
    def report(self): 
        rprt_dict = {k: (self.beta_hat[k], self.zs[k]) for k in range(self.p)}
        s = ''
        for k in range(self.p):
            s += f'Coeff #{k} is {rprt_dict[k][0]:.3}, garnering a z-score of {rprt_dict[k][1]:.3}\n'
        return s

In [19]:
ols = OLS(df_unnormalized, 'lpsa')

print(ols.report())

Coeff #0 is 2.48, garnering a z-score of 48.8
Coeff #1 is 0.784, garnering a z-score of 8.98
Coeff #2 is 0.114, garnering a z-score of 4.33
Coeff #3 is -1.18, garnering a z-score of -2.68
Coeff #4 is 0.204, garnering a z-score of 2.33
Coeff #5 is 0.131, garnering a z-score of 4.41
Coeff #6 is -0.207, garnering a z-score of -1.65
Coeff #7 is 0.0257, garnering a z-score of 0.443
Coeff #8 is 3.55, garnering a z-score of 1.43



# to be continued.... the F-score function

### it basically looks like this: 

$$F = \frac{(RSS_0 - RSS_1) * (N - p_1 - 1)}{(p_1 - p_0) * RSS_1} $$ 

where $RSS_1$ is the loss of a _larger_ model, i.e., a model with $p_1$ coefficients; and $RSS_0$ is a _smaller_ model with $p_0$ coefficients (we require $p_1 > p_0$ )


## _Exercise 3.1_ states that $z_i ^ 2 == F$ when $RSS_1$ is the full model and $RSS_0$ is the model having dropped feature #$i$ (or required that it's coefficient is exactly $0$). 